# Understanding Containerization
### Docker Containers: Fundamentals, Security, and Hands-On Labs

## Prerequisites:

- Basic understanding of command-line interfaces (Linux/bash or Windows PowerShell preferred)
- Basic familiarity with virtualization concepts (desirable)

## Software:

- Docker Desktop installed (or another compatible Docker environment)

## Agenda

1. Introduction & Core Concepts
    - Lab 1: Basic Container Operations
3. Building Images and Dockerfiles
    - Lab 2: Creating a Dockerfile 
    - Lab 3: Docker Compose for Multi-Container App
4. Docker Security 
    - Image Security
    - Container Runtime Security
    - Demo: Compromise Emulation
5. Networking and Storage
    - Port Mapping
    - Docker Volumes
    - Lab 4: Communicating Containers
6. Conclusions and what's next






## Lab 1 - Basic Container Operations
###  See Docker CLI cheat sheet here: https://docs.docker.com/get-started/docker_cheatsheet.pdf

1. Pulling an Image

    -  Open a terminal or command prompt.

    -  Run the command: `docker pull nginx`

2. Running a Container

    - Run the command: `docker run -d -p 80:80 nginx`

    - **Explanation:** 

        - `-d` runs the container in detached mode (background)

        - `-p` 80:80 maps port 80 of the container to port 80 on your host machine.

        - The official Docker ngnix image can be found here: https://hub.docker.com/_/nginx 

3. Verifying the Container

    -   Open a web browser and go to `http://localhost`. You should see the Nginx welcome page.

    -   In your terminal, type: `docker ps` to list running containers.

4. Stopping and Removing

    - Get the container ID from `docker ps`.

    - Stop the container: docker stop <container_id>
    
    - Remove the container: docker rm <container_id>

    - Bonus command-line tricks for cleaning up unused containers and freeing up disk space:

        - `docker ps -a -q`: This command lists all containers, including stopped ones (-a flag), and extracts only their container IDs (-q flag). It provides a list of container IDs that have exited or are currently stopped.
        
        - `docker stop $(docker ps -a -q)`: Stops all running containers
        
        - `docker rm $(docker ps -a -q)`: Removes all containers

## Lab 2 - Building a Custom Docker Image for a Web App

1. Create a Working Directory

    - Open your terminal or command prompt.

    - Create a new directory for this exercise (e.g., docker-training): `mkdir docker-training`

    - Change into that directory: `cd docker-training`


2. Create a Simple Web Application:

    - Inside the docker-training directory, create an index.html file with some basic content (e.g., “Hello, Docker!”).
    
        `echo "Hello, Docker" > index.html`
        

3. Create a Dockerfile

    - Create a Dockerfile (case-sensitive) in the same directory: `touch Dockerfile`

    - Use a text editor or code editor to edit Dockerfile:

        ```
        # Dockerfile
        FROM nginx
        COPY index.html /usr/share/nginx/html
        ```
<br>

4. Build the Custom Image

    - Build the Docker image using the Dockerfile: `docker build -t my-web-app .` (Don't forget the period at the end).

    - **Explanation**:

        - `-t my-web-app`: The -t option stands for "tag". This part of the command tags the resulting image with a name and optionally a tag in the 'name:tag' format. If you don't specify a tag, Docker will assign the latest tag by default. Here, my-web-app is the name you're giving to the Docker image. This makes it easier to reference the image later, for example, when you want to run a container based on this image.

        - `.`: This specifies the context for the build. In Docker, the build context is the set of files located in the specified PATH or URL. The Docker build process can use any files in the context to build the image. In this case, . means the current directory you're in when you run the command. Docker will look for a Dockerfile in this directory to build the image. If your Dockerfile is named differently or located elsewhere, you would need to use the -f option to specify the path to the Dockerfile.


5. Run the Container

    - Start a container from the custom image: `docker run -d -p 8080:80 my-web-app`

    - Open a web browser and go to `http://localhost:8080`

    <br>

6. Running Commands Inside the Container

    - Check what user the container is running as: `docker exec -it <container_id> whoami`

    - **Explanation**:

        - `docker exec`: This is the main command that tells Docker to execute a specific command inside a running container.
        
        - `it`: This option is a combination of -i and -t flags:
        
            - `i` (or --interactive) keeps STDIN open even if not attached. This allows you to interact with the command if necessary.

            - `t` (or --tty) allocates a pseudo-TTY, which means it simulates a terminal, similar to what you would get if you were to log into a session on the container. This is useful for commands that require a terminal to operate properly, like shell sessions.

        - `whoami`: This is the command to be executed inside the container. whoami is a UNIX command that displays the username of the current user. When run in the context of this Docker command, it will display the username of the user inside the container, not the host machine.



## Lab 3 - Docker Compose for Multi-Container App

1. Create a Docker Compose File

    - In the same docker-training directory, create a docker-compose.yml file: `touch docker-compose.yml`

    - Use a text editor or code editor to edit docker-compose.yml:

        ```
        version: '3'
        services:
            web:
                image: nginx:alpine
                ports:
                    - "8081:80"
            db:
                image: postgres:latest
                environment:
                    POSTGRES_USER: myuser
                    POSTGRES_PASSWORD: mypassword
        ```

         - **Explanation**:

            - `version: '3'`: This specifies the version of the Docker Compose file format. Version 3 is suitable for running with Docker Engine version 1.13.0+ and Docker Compose 1.10.0+. It supports defining services, networks, and volumes at a high level.

            - The services key is used to configure the containers that make up your application. Here, two services are defined: web and db.

            - `image`: `nginx:alpine`: This line specifies that the web service uses the nginx:alpine image. This is a lightweight version of the Nginx web server packaged in the Alpine Linux distribution. `postgres:latest`: This line specifies that the db service uses the latest version of the postgres image. This container will run a PostgreSQL database server.

            - `ports`: This section maps ports between the host and the container, allowing external access to services on a container.
            
            - `"8081:80"`: This maps port 8081 on the host to port 80 on the container. This means that the Nginx server inside the container, which listens on port 80, can be accessed on port 8081 from the host machine.

            - `environment`: This section allows you to set environment variables inside the container. For the db service, two environment variables are set: 
            
                - `POSTGRES_USER: myuser`: Sets the default username for the PostgreSQL database.
                - `POSTGRES_PASSWORD: mypassword`: Sets the password for the PostgreSQL database. <br>

            This Docker Compose file sets up a simple web application environment with two services: an Nginx web server and a PostgreSQL database. The Nginx server is accessible from the host machine on port 8081, and the PostgreSQL database is configured with a specified username and password. This setup is useful for development and testing environments where you need a straightforward web server and database setup.


2. Start the Services

    - Run the services defined in the docker-compose.yml file: `docker-compose up -d` <br>

3. Access the Web Server:

    - Open a web browser and visit `http://localhost:8081`

    - You should see the default Nginx page.

4. Check Resource Limits

    - Inspect the running containers: `docker stats`
    

## Basic Container Communication



1. Create Two Simple Apps

    - We’ll create two simple Python applications, a Web Server (Nginx) that will serve a static website, and a backend API application (FastAPI) that will provide data to the web server.

    - App 1: Python Web Server

        1. Create a new directory within your docker-training directory: `mkdir web_server`

        2. Change into that directory: `cd web_server`

        3. Use a text editor or code editor to create `web_server.py` with these content:
        
            ``` python
            # web_server.py
            from flask import Flask

            app = Flask(__name__)

            @app.route("/")
            def hello():
                return "Hello from the web server!"

            if __name__ == "__main__":
                app.run(host="0.0.0.0", port=8082)
            ```
        <br>

    - App 2: Backend API

        1. Create a new directory within your docker-training directory: `mkdir backend_api`

        2. Change into that directory: `cd backend_api`

        3. Use a text editor or code editor to create `backend_api.py` with these content:

            ``` python
            # backend_api.py
            from fastapi import FastAPI

            app = FastAPI()

            @app.get("/movies/random")
            def get_random_movie():
                return {"movie": "Inception"}

            if __name__ == "__main__":
                import uvicorn
                uvicorn.run(app, host="0.0.0.0", port=8001)
            ```
        <br>

2. Create Dockerfiles for each application inside web_server and backend_api directories

 - Use a text editor or code editor to create and edit Dockerfile inside the web_server directory:

    ```
    # Dockerfile for web_server
    FROM nginx:alpine
    COPY web_server.py /usr/share/nginx/html/index.html
    ```
    <br>
 - Use a text editor or code editor to create and edit Dockerfile inside the backend_api directory:

    ```
    # Dockerfile for backend_api
    FROM python:3.9
    COPY backend_api.py /app/backend_api.py
    WORKDIR /app
    RUN pip install fastapi uvicorn
    CMD ["uvicorn", "backend_api:app", "--host", "0.0.0.0", "--port", "8001"]
    ```
    <br>

3. Build Docker Images for Both Applications (Note you have to be in the parent directory)

    - Exit to the parent directory (docker-training): `cd ..`
    - `docker build -t my_web_server ./web_server`
    - `docker build -t my_backend_api ./backend_api`
    <br>

4. Create a Custom Docker Network

    - Create a custom bridge network to allow communication between containers: `docker network create my_app_network`

5. Configure Communication

    - Inside the web server container, configure it to make requests to the backend API using the container name (DNS resolution within the network).
    
    - Update your web server configuration to point to http://backend_api_container:8001.

6. Demonstrate Basic Container Communication

    - Access the web server from your browser or use curl: `curl http://localhost:8082`

7. Cleanup 

    - When done, stop and remove the containers: `docker stop web_server_container backend_api_container` and `docker rm web_server_container backend_api_container`

    - Also, remove the custom network: `docker network rm my_app_network`

    

## 


Snyk - free account

https://docs.snyk.io/snyk-cli/authenticate-the-cli-with-your-account

Authenticate with Docker ID and authorize the App

    - docker scan --login

Once authenticated you're ready to scan with the free API

    - docker scan my-web-app

## Emulating a Compromise

docker run -it --rm --name test_container1 centos /bin/bash

[root@aa8f8e36d07a /]# SECRET=SuperSecretPassword1234

[root@aa8f8e36d07a /]# echo $SECRET

